<a href="https://colab.research.google.com/github/arimbawa/Aplikasi-IoT/blob/main/Minggu_5_API_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask flask-ngrok mongita
!pip install pyngrok

In [11]:
# Konfigurasi Ngrok dengan token Agus
from pyngrok import ngrok

NGROK_TOKEN = "xxxxxxxxxxxxx"
ngrok.set_auth_token(NGROK_TOKEN)


In [ ]:
from flask import Flask, request, jsonify
from mongita import MongitaClientDisk

# Setup Flask
app = Flask(__name__)

# Setup Mongita (lokal, mirip MongoDB tapi embedded)
client = MongitaClientDisk()
db = client["iot_database"]
collection = db["sensor_data"]

@app.route('/sensor', methods=['POST'])
def receive_sensor():
    data = request.json
    collection.insert_one(data)
    return jsonify({"status": "success", "inserted": data})

@app.route('/data', methods=['GET'])
def get_data():
    docs = []
    for d in collection.find({}):
        d.pop("_id", None)  # hapus _id biar bisa di-JSON-kan
        docs.append(d)
    return jsonify(docs)

# Jalankan Flask via Ngrok
public_url = ngrok.connect(5000)
print("Flask server running on:", public_url)

app.run(port=5000)

In [15]:
from flask import Flask, request, jsonify
from mongita import MongitaClientDisk
from pyngrok import ngrok
from bson import ObjectId

# Setup Flask
app = Flask(__name__)

# Setup Mongita
client = MongitaClientDisk()
db = client["iot_database"]
collection = db["sensor_data"]

# Helper untuk convert ObjectId ke string
def clean_doc(doc):
    doc["_id"] = str(doc["_id"])
    return doc

# ---------- CREATE ----------
@app.route('/sensor', methods=['POST'])
def create_sensor():
    data = request.json
    result = collection.insert_one(data)
    return jsonify({"status": "success", "id": str(result.inserted_id)})

# ---------- READ (All) ----------
@app.route('/sensor', methods=['GET'])
def read_all_sensors():
    docs = [clean_doc(d) for d in collection.find({})]
    return jsonify(docs)

# ---------- READ (By ID) ----------
@app.route('/sensor/<id>', methods=['GET'])
def read_sensor(id):
    doc = collection.find_one({"_id": ObjectId(id)})
    if doc:
        return jsonify(clean_doc(doc))
    else:
        return jsonify({"error": "Data not found"}), 404

# ---------- UPDATE ----------
@app.route('/sensor/<id>', methods=['PUT'])
def update_sensor(id):
    data = request.json
    result = collection.update_one({"_id": ObjectId(id)}, {"$set": data})
    if result.matched_count:
        return jsonify({"status": "updated", "id": id})
    else:
        return jsonify({"error": "Data not found"}), 404

# ---------- DELETE ----------
@app.route('/sensor/<id>', methods=['DELETE'])
def delete_sensor(id):
    result = collection.delete_one({"_id": ObjectId(id)})
    if result.deleted_count:
        return jsonify({"status": "deleted", "id": id})
    else:
        return jsonify({"error": "Data not found"}), 404

# ---------- RUN SERVER ----------
public_url = ngrok.connect(5000)
print("Flask server running on:", public_url)
app.run(port=5000)

Flask server running on: NgrokTunnel: "https://47da96db7286.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 12:24:05] "POST /sensor HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 12:26:09] "GET /sensor HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 12:27:13] "GET /sensor/68cd46d4600cc1fbd64865f6 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 12:28:02] "PUT /sensor/68cd46d4600cc1fbd64865f6 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 12:28:09] "GET /sensor/68cd46d4600cc1fbd64865f6 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 12:28:19] "PUT /sensor/68cd46d4600cc1fbd64865f6 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 12:28:22] "GET /sensor/68cd46d4600cc1fbd64865f6 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2025 12:28:53] "DELETE /sensor/68cd46d4600cc1fbd64865f6 HTTP/1.1" 200 -
IN